In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm

In [3]:
print("Đang tải dữ liệu đã chuẩn bị từ file CSV...")
try:
    df_market = pd.read_csv("market_data_snapshot.csv")
    df_funda_2021 = pd.read_csv("funda_data_2021.csv")
    print("--- Tải file thành công ---")
except FileNotFoundError:
    print("LỖI: Không tìm thấy file 'market_data_snapshot.csv' hoặc 'funda_data_2021.csv'.")
    print("Vui lòng CHẠY file input_data.ipynb TRƯỚC TIÊN để tạo ra các file này.")
    # Bạn có thể thêm raise để dừng thực thi nếu muốn
# -----------------------

print("\n--- BẮT ĐẦU: Gộp, Tính toán và Tiền xử lý Dữ liệu ---")
# Code GỘP (merge) của bạn giờ sẽ hoạt động
df_matrix = pd.merge(df_market, df_funda_2021, on='ticker', how='left')

print("Đang tính toán các chỉ số phái sinh...")

Đang tải dữ liệu đã chuẩn bị từ file CSV...
--- Tải file thành công ---

--- BẮT ĐẦU: Gộp, Tính toán và Tiền xử lý Dữ liệu ---
Đang tính toán các chỉ số phái sinh...


In [4]:
print("\n--- BẮT ĐẦU: Gộp, Tính toán và Tiền xử lý Dữ liệu ---")
df_matrix = pd.merge(df_market, df_funda_2021, on='ticker', how='left')

print("Đang tính toán các chỉ số phái sinh...")

col_tassets = 'Total Assets'
col_equity = "Total Shareholder's Equity"
col_ti = 'Total Income'
col_loans = 'Total Loans'
col_deposits = 'Total Deposits'
col_roe = 'Returns Over Equity'
col_roa = 'Returns Over Assets'
col_nim = 'Net Interest Margin'
col_cir = 'Cost-Income Ratios'
col_npl = 'Non-performing Loans Ratio'
col_car = 'Capital Adequacy Ratio' 
cols_to_numeric = [
    col_tassets, col_equity, col_ti, col_loans, col_deposits, 
    col_roe, col_roa, col_nim, col_cir, col_npl, col_car
]
for col in cols_to_numeric:
    if col in df_matrix.columns:
        df_matrix[col] = pd.to_numeric(df_matrix[col], errors='coerce')

df_matrix['D_E'] = (df_matrix[col_tassets] - df_matrix[col_equity]) / df_matrix[col_equity]
df_matrix['LDR'] = (df_matrix[col_loans] / df_matrix[col_deposits])
df_matrix['Asset_Turnover'] = (df_matrix[col_ti] / df_matrix[col_tassets])

print("Đang tiền xử lý (lọc cột, loại bỏ 'nan')...")

final_criteria_columns_map = {
    'ticker': 'ticker',
    'P/E (TTM)': 'P/E (TTM)',
    'P/B': 'P/B',
    'EPS (TTM)': 'EPS (TTM)',
    'ROE': col_roe,
    'ROA': col_roa,
    'D_E': 'D_E',
    'CAR': col_car,
    'LDR': 'LDR',
    'NPL_Ratio': col_npl,
    'Asset_Turnover': 'Asset_Turnover',
    'NIM': col_nim,
    'CIR': col_cir
}
available_cols = [col for col in final_criteria_columns_map.values() if col in df_matrix.columns]
df_decision_matrix = df_matrix[available_cols].copy()

df_decision_matrix.rename(columns={v: k for k, v in final_criteria_columns_map.items()}, inplace=True)

initial_rows = len(df_decision_matrix)
df_final_matrix = df_decision_matrix.dropna().reset_index(drop=True)
final_rows = len(df_final_matrix)

print(f"Đã loại bỏ {initial_rows - final_rows} hàng do thiếu dữ liệu (MBB, TPB).")

print("\n--- HOÀN TẤT: MA TRẬN QUYẾT ĐỊNH CUỐI CÙNG SẴN SÀNG CHO TOPSIS ---")
display(df_final_matrix)
df_final_matrix.to_csv("DECISION_MATRIX_FOR_TOPSIS.csv", index=False)


--- BẮT ĐẦU: Gộp, Tính toán và Tiền xử lý Dữ liệu ---
Đang tính toán các chỉ số phái sinh...
Đang tiền xử lý (lọc cột, loại bỏ 'nan')...
Đã loại bỏ 2 hàng do thiếu dữ liệu (MBB, TPB).

--- HOÀN TẤT: MA TRẬN QUYẾT ĐỊNH CUỐI CÙNG SẴN SÀNG CHO TOPSIS ---


,ticker,P/E (TTM),P/B,EPS (TTM),ROE,ROA,D_E,LDR,NPL_Ratio,Asset_Turnover,NIM,CIR
0,VCB.VN,22.498590,2.232644,2644.61,20.11,1.52,12.202737,0.846234,0.64,0.054072,9.00,64.84
1,BID.VN,13.332171,1.652211,2869.00,12.56,0.62,19.406761,0.974815,1.01,0.053206,11.55,85.55
2,CTG.VN,7.925258,1.565975,6208.00,15.18,0.93,15.354462,0.973163,1.26,0.054418,10.70,78.90
3,OCB.VN,8.481765,1.011455,1461.96,20.20,2.39,7.460947,1.032854,1.32,0.076545,7.04,60.92
